## Using Komo for IK

In [ ]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry

In [ ]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
D = C.view()

In [ ]:
obj = C.addFrame("object")
obj.setPosition([1., 0., 1.5])
obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])

In [ ]:
IK = C.komo_IK(False)
IK.add_qControlObjective([], 1, 1.)
IK.addObjective([1.], 
                ry.FS.positionRel, 
                ["object", "R_gripper"], 
                ry.OT.eq, 
                [1e2],
                [0.,0.,-.2]);

In [ ]:
# Calling the optimizer (True means random initialization/restart)
IK.optimize()
IK.getReport()

In [ ]:
C.setFrameState( IK.getFrameState(0) )

In [ ]:
# Move object and reoptimize

# move object
obj.setPosition([.2,.2,1.5])

# # copy C into the IK's internal configuration(s)
# IK.setConfigurations(C)

# redefine the IK problem
IK = C.komo_IK(False)
IK.add_qControlObjective([], 1, 1.)
IK.addObjective([1.], 
                ry.FS.positionRel, 
                ["object", "R_gripper"], 
                ry.OT.eq, 
                [1e2],
                [0.,0.,-.2]);
# reoptimize
IK.optimize(0.) # 0 indicates: no adding of noise for a random restart
print(IK.getReport())

# grab result
C.setFrameState( IK.getFrameState(0) )

## Path Optimization

In [ ]:
obj.setPosition([.8,.2,1.5])

# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,40, 5., True)
komo.addObjective([1.], ry.FS.positionDiff, ["R_gripper", "object"], ry.OT.sos, [1e2])
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)
komo.addObjective([], ry.FS.distance, ["R_gripper", "object"], ry.OT.ineq, [1e2])

komo.optimize()
komo.getReport()

In [ ]:
V = komo.view_play(True, 0.1)

In [ ]:
C.setFrameState(komo.getFrameState(19))
C.getJointState()
coll = C.feature(ry.FS.accumulatedCollisions, [])
C.computeCollisions() 
coll.eval(C)